# Climate Velocity functions

In [ ]:
# Import necessary packages
from shapely.geometry import Polygon, Point, MultiPolygon
import shapefile
import geopandas as gpd
import xarray as xr
import time
import numpy as np
from scipy.stats import linregress
from scipy.interpolate import interp1d as int1
import matplotlib.pyplot as plt
import getpass
import pandas as pn
%matplotlib inline

In [ ]:
# Login information for the Copernicus Marine
USERNAME = 'your_username_here'
PASSWORD = getpass.getpass('Enter your password: ')
DATASET_ID = 'cmems_mod_glo_phy_my_0.083_P1M-m'


In [ ]:
# Function to retrieve data from copernicus
def copernicusmarine_datastore(dataset, username, password):
    from pydap.client import open_url
    from pydap.cas.get_cookies import setup_session
    cas_url = 'https://cmems-cas.cls.fr/cas/login'
    session = setup_session(cas_url, username, password)
    session.cookies.set("CASTGC", session.cookies.get_dict()['CASTGC'])
    database = ['my', 'nrt']
    url = f'https://{database[0]}.cmems-du.eu/thredds/dodsC/{dataset}'
    try:
        data_store = xr.backends.PydapDataStore(open_url(url, session=session))#, user_charset='utf-8')) # needs PyDAP >= v3.3.0 see https://github.com/pydap/pydap/pull/223/commits 
    except:
        url = f'https://{database[1]}.cmems-du.eu/thredds/dodsC/{dataset}'
        data_store = xr.backends.PydapDataStore(open_url(url, session=session))#, user_charset='utf-8')) # needs PyDAP >= v3.3.0 see https://github.com/pydap/pydap/pull/223/commits
    return data_store

In [ ]:
# Retrieve Data
data_store = copernicusmarine_datastore(DATASET_ID, USERNAME, PASSWORD)

In [ ]:
# Take a look at the dataset
DS = xr.open_dataset(data_store)
DS

In [ ]:
# Loading the LME datafile
LME = gpd.read_file('/Users/nyelab/Downloads/LME66/LMEs66.shp')

## Functions for calculating climate velocity

### The final function **climate_vel_subsets_more** includes all the functions listed below.  You input an LME and a filename and directory for your output files and vertical and horizontal climate velocities are calculated and saved to that directory

**find_nearest** and **LME_data_retreval_2** are used to subset the temperature data to the LME of intereste

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
def LME_data_retreval_2(LME_name):
    lme = LME[LME.LME_NAME  == LME_name]
    bounds = lme.geometry.apply(lambda x: x.bounds).tolist()
    lon_min=bounds[0][0]
    lat_min=bounds[0][1]
    lon_max=bounds[0][2]
    lat_max=bounds[0][3]
    if lon_min <-100 and lon_max > 100:
        lon_min_ind = find_nearest(totalLON, lon_min) + 5
        lon_max_ind = find_nearest(totalLON, lon_max) - 5
    else:
        lon_min_ind = find_nearest(totalLON, lon_min) - 5
        lon_max_ind = find_nearest(totalLON, lon_max) + 5
    if lat_min < -79.58332059999996:
        lat_min_ind = 0
        lat_max_ind = find_nearest(totalLAT, lat_max) + 5
    if lat_max > 89.5885158000143:
        lat_min_ind = find_nearest(totalLAT, lat_min) - 5
        lat_max_ind = 2040
    if lat_min >= -79.58332059999996 and lat_max <= 89.5885158000143:
        lat_min_ind = find_nearest(totalLAT, lat_min) - 5
        lat_max_ind = find_nearest(totalLAT, lat_max) + 5
    data = DS.thetao[:,:27,lat_min_ind:lat_max_ind,lon_min_ind:lon_max_ind]
    
    return data

**inSHAPE_all_depth_glorys** replaces all temperature data outside of the LME with nans

In [ ]:
def inSHAPE_all_depth_glorys(longitude, latitude, var, depth, shape):
    x = longitude
    y = latitude
    empty = np.empty([28, len(depth), len(y), len(x)])
    empty[:,:,:,:] = np.nan
    for i in range(len(y)):
        for j in range(len(x)):
            if Point(x[j], y[i]).within(shape) == True:
                empty[:,:,i,j] = var[:,:,i,j]
    return empty

**interpolate_5m_2** linearly interpolates the temperature data at 5m spacing in the upper 200m

In [ ]:
def interpolate_5m_2(tempdata, LME_Name):
    start = time.perf_counter()
    
    datayr = tempdata.groupby('time.year').mean('time')
    
    time2 = time.perf_counter()
    print('Yearly averaging completed, '+ str((time2-start)/60) + ' minutes elapsed')
    
    temp = inSHAPE_all_depth_glorys(datayr.longitude, datayr.latitude, datayr.thetao, datayr.depth,
                             LME[LME.LME_NAME == LME_Name].geometry[LME.OBJECTID[LME.LME_NAME == LME_Name].values[0] -1])
    
    time3 = time.perf_counter()
    print('Subsetting to LME area completed, '+ str((time3-start)/60) + ' minutes elapsed')
    
    z = np.arange(5,205,5)
    
    temp_5m = np.empty([28,len(z),len(datayr.latitude),len(datayr.longitude)])
    temp_5m[:,:,:,:] = np.nan
    for t in range(28):
        for i in range(len(datayr.latitude)):
            for j in range(len(datayr.longitude)):
                Y_data = temp[t,:,i,j]
                if np.any(Y_data)==True:
                    func = int1(datayr.depth,Y_data,bounds_error = False)
                    temp_5m[t,:,i,j] = func(z)
                    
    time4 = time.perf_counter()
    print('5m interpolation completed, '+ str((time4-start)/60) + ' minutes elapsed')
    
    temp5m = xr.Dataset(data_vars = {'temp': (['year','depth','lat','lon'],temp_5m)}, 
                    coords = {'year': datayr.year, 'depth': z, 'lat': np.array(datayr.latitude), 'lon': np.array(datayr.longitude)})
    return temp5m


**temptrend_vcv** calculates the slope and pvals of any significant linear trend of temperature at each gridpoint within the LME

In [ ]:
def temptrend_vcv(temp5m):
    
    start = time.perf_counter()
    
    slopes = np.empty([len(temp5m.lat),len(temp5m.lon),len(temp5m.depth)])
    p_vals = np.empty([len(temp5m.lat),len(temp5m.lon),len(temp5m.depth)])
    slopes[:,:,:] = np.nan
    p_vals[:,:,:] = np.nan

    x = np.arange(0,27)
    for i in range(len(temp5m.lat)):
        for j in range(len(temp5m.lon)):
            for k in range(len(temp5m.depth)):
                y = temp5m.temp[:27,k,i,j]
                if any(y) == True:
                    LR = linregress(x, y)
                    slopes[i,j,k] = LR.slope
                    p_vals[i,j,k] = LR.pvalue
    p_vals[p_vals>0.05] = np.nan                
    time4 = time.perf_counter()
    
    print('Temperature Trends Complete, '+ str((time4-start)/60) + ' minutes elapsed')
    
    return slopes, p_vals

**vertical_spat_grad** calculates the vertical spatial gradient

In [ ]:
def vertical_spat_grad(temp_data):
# Calculate the spatial gradient in the vertical
    start = time.perf_counter()
    
    UP = 5 # 5 meters between each data point
    DOWN = 5

    temp_slice_mean = np.nanmean(temp_data.temp[:,:,:,:],0)
    UD_net = np.empty([len(temp_data.depth),len(temp_data.lat),len(temp_data.lon)])
    UD_net[:,:,:] = np.nan

    for i in range(len(temp_data.depth)):
        for j in range(len(temp_data.lat)):
            for k in range(len(temp_data.lon)):
                if np.isnan(temp_slice_mean[i,j,k]) == False:
                    if i == 0: #at the surface
                        focal_temp = temp_slice_mean[i,j,k]
                        DO_grad = -1*(focal_temp - temp_slice_mean[i+1,j,k])/DOWN # angle = 0
                        UD_net[i,j,k] = DO_grad
                    if i == (len(temp_data.depth) -1): #at the bottom
                        focal_temp = temp_slice_mean[i,j,k]
                        UP_grad = (focal_temp - temp_slice_mean[i-1,j,k])/UP # angle = 180
                        UD_net[i,j,k] = UP_grad
                    if i >0 and i < (len(temp_data.depth)-1):
                        focal_temp = temp_slice_mean[i,j,k]
                        UP_grad = (focal_temp - temp_slice_mean[i-1,j,k])/UP # angle = 0
                        DO_grad = -1*((focal_temp - temp_slice_mean[i+1,j,k])/DOWN) # angle = 180
                        if np.isnan(UP_grad) == True and np.isnan(DO_grad) == True:
                            UD_net[i,j,k] = np.nan
                        else:
                            UD_net[i,j,k] = np.nansum([UP_grad, DO_grad])/2

    time4 = time.perf_counter()
    
    print('Vertical Spat Grad Complete, '+ str((time4-start)/60) + ' minutes elapsed')
    
    return UD_net

**temptrend_hcv** calculates the slope and pvals of any significant linear trend of temperature at each gridpoint within the LME using only the original uppermost bin (closest to the surface)

In [ ]:
def temptrend_hcv(temp):
    
    slopes = np.empty([len(temp.lat),len(temp.lon)])
    p_vals = np.empty([len(temp.lat),len(temp.lon)])
    slopes[:,:] = np.nan
    p_vals[:,:] = np.nan

    x = np.arange(0,27)
    for i in range(len(temp.lat)):
        for j in range(len(temp.lon)):
            y = temp.temp[:27,i,j]
            if any(y) == True:
                LR = linregress(x, y)
                slopes[i,j] = LR.slope
                p_vals[i,j] = LR.pvalue
                    
    p_vals[p_vals>0.05] = np.nan
    return slopes, p_vals

**horiz_spatgrad_hcv** calculates the horizontal spatial gradient in the method of Burrows et al. (2011)

In [ ]:
def horiz_spatgrad_hcv(netcdf):

    slice_mean = np.nanmean(netcdf.temp[:,:,:],0)
    shape = np.shape(slice_mean)
    spatgrad_NS = np.empty(shape)
    spatgrad_NS[:,:] = np.nan
    spatgrad_EW = np.empty(shape)
    spatgrad_EW[:,:] = np.nan
    spatgrad = np.empty(shape)
    spatgrad[:,:] = np.nan

    for j in range(len(netcdf.lat)):
        for k in range(len(netcdf.lon)):
            if np.isnan(slice_mean[j,k]) == False:
                if j==0:
                    spatgrad_NS[j,k] = np.nan
                    spatgrad_EW[j,k] = np.nan
                if j==(len(netcdf.lat)-1):
                    spatgrad_NS[j,k] = np.nan
                    spatgrad_EW[j,k] = np.nan
                if k==0:
                    spatgrad_NS[j,k] = np.nan
                    spatgrad_EW[j,k] = np.nan
                if k==(len(netcdf.lon)-1):
                    spatgrad_NS[j,k] = np.nan
                    spatgrad_EW[j,k] = np.nan
                    
                focal_temp = slice_mean[j,k]
                if j > 0 and j < (len(netcdf.lat)-1) and k >0 and k< (len(netcdf.lon)-1):
                    north_temp = slice_mean[j+1,k]
                    south_temp = slice_mean[j-1,k]
                    east_temp = slice_mean[j,k+1]
                    west_temp = slice_mean[j,k-1]
                    north_east_temp = slice_mean[j+1, k+1]
                    north_west_temp = slice_mean[j+1, k-1]
                    south_east_temp = slice_mean[j-1, k+1]
                    south_west_temp = slice_mean[j-1, k-1]
                
                    gradWE1 = (north_temp - north_west_temp)/((np.cos(np.radians(netcdf.lat[j+1]))*111.321)*(1/12))
                    gradWE2 = (focal_temp-west_temp)/((np.cos(np.radians(netcdf.lat[j]))*111.321)*(1/12))
                    gradWE3 = (south_temp - south_west_temp)/((np.cos(np.radians(netcdf.lat[j-1]))*111.321)*(1/12))
                    gradWE4 = (north_east_temp - north_temp)/((np.cos(np.radians(netcdf.lat[j+1]))*111.321)*(1/12))
                    gradWE5 = (east_temp-focal_temp)/((np.cos(np.radians(netcdf.lat[j]))*111.321)*(1/12))
                    gradWE6 = (south_east_temp - south_temp)/((np.cos(np.radians(netcdf.lat[j-1]))*111.321)*(1/12))
                
                    gradNS1 = (north_west_temp-west_temp)/(111.2*1/12)
                    gradNS2 = (north_temp-focal_temp)/(111.2*1/12)
                    gradNS3 = (north_east_temp - east_temp)/(111.2*1/12)
                    gradNS4 = (west_temp - south_west_temp)/(111.2*1/12)
                    gradNS5 = (focal_temp-south_temp)/(111.2*1/12)
                    gradNS6 = (east_temp - south_east_temp)/(111.2*1/12)
            
                    spatgrad_EW[j,k] = np.average([gradWE1,gradWE2,gradWE3,gradWE4,gradWE5,gradWE6], weights = [1,2,1,1,2,1])
                    spatgrad_NS[j,k] = np.average([gradNS1,gradNS2,gradNS3,gradNS4,gradNS5,gradNS6], weights = [1,2,1,1,2,1])
                
                    spatgrad[j,k] = np.sqrt((spatgrad_EW[j,k]**2)+(spatgrad_NS[j,k]**2))

    return spatgrad_EW, spatgrad_NS, spatgrad

**VCV_HCV** combines the outputs of all the previous functions and returns xr datasets with all the vertical and horizontal climate velocity data

In [ ]:
def VCV_HCV(VSG,TT5m,PV5m,temp5m, temp_yr, TT_yr, PV_yr, sg, ew_sg, ns_sg):
    
    vcv = xr.Dataset(data_vars = {'temp': (['year','depth','lat','lon'],temp5m.temp),
                                 'v_spatgrad': (['depth','lat','lon'], VSG),
                                 'temptrend': (['lat','lon','depth'], TT5m),
                                  'pvals': (['lat','lon','depth'], PV5m),
                                 'VCV': (['lat','lon','depth'],TT5m/(VSG.transpose(1,2,0)))},
                     coords = {'year': temp5m.year, 'depth': temp5m.depth, 
                               'lat': np.array(temp5m.lat), 
                               'lon': np.array(temp5m.lon)})
    
    hcv = xr.Dataset(data_vars = {'temp': (['year','lat','lon'],temp_yr.temp),
                                  'ew_spatgrad': (['lat','lon'], ew_sg),
                                  'ns_spatgrad': (['lat','lon'], ns_sg),
                                  'horiz_spatgrad': (['lat','lon'],sg),
                                  'temptrend': (['lat','lon'], TT_yr),
                                  'pvals': (['lat','lon'], PV_yr),
                                  'HCV': (['lat','lon'], TT_yr/sg)},
                     coords = {'year': temp_yr.year, 
                               'lat': np.array(temp_yr.lat), 
                               'lon': np.array(temp_yr.lon)})
    return vcv, hcv

**climate_vel_subsets_more** uses all previous functions to create and save vertical and horizontal climate velocity

In [ ]:
def climate_vel_subsets_more(LME_name, file_name, directory):
    
    data = LME_data_retreval_2(LME_name)
    location = directory + file_name + '1.nc'
    data[:,:3,:,:].to_netcdf(location)
    
    data = LME_data_retreval_2(LME_name)
    location = directory + file_name + '2.nc'
    data[:,3:6,:,:].to_netcdf(location)
    
    data = LME_data_retreval_2(LME_name)
    location = directory + file_name + '3.nc'
    data[:,6:9,:,:].to_netcdf(location)
    
    data = LME_data_retreval_2(LME_name)
    location = directory + file_name + '4.nc'
    data[:,9:12,:,:].to_netcdf(location)
    
    data = LME_data_retreval_2(LME_name)
    location = directory + file_name + '5.nc'
    data[:,12:15,:,:].to_netcdf(location)
    
    data = LME_data_retreval_2(LME_name)
    location = directory + file_name + '6.nc'
    data[:,15:18,:,:].to_netcdf(location)
    
    data = LME_data_retreval_2(LME_name)
    location = directory + file_name + '7.nc'
    data[:,18:21,:,:].to_netcdf(location)
    
    data = LME_data_retreval_2(LME_name)
    location = directory + file_name + '8.nc'
    data[:,21:,:,:].to_netcdf(location)
    
    data1 = xr.open_dataset(directory + file_name + '1.nc')
    data2 = xr.open_dataset(directory + file_name + '2.nc')
    data3 = xr.open_dataset(directory + file_name + '3.nc')
    data4 = xr.open_dataset(directory + file_name + '4.nc')
    data5 = xr.open_dataset(directory + file_name + '5.nc')
    data6 = xr.open_dataset(directory + file_name + '6.nc')
    data7 = xr.open_dataset(directory + file_name + '7.nc')
    data8 = xr.open_dataset(directory + file_name + '8.nc')
    
    data = xr.merge([data1,data2,data3,data4,data5,data6,data7,data8])
    
    LME_Name = LME_name
    # Vertical Climate Velocity
    # interpolate to 5m
    data5m = interpolate_5m_2(data, LME_name)
    
    #calculate the temperature trend
    slopes, pvals = temptrend_vcv(data5m)
    
    #calculate the spatial gradient
    spatgrad_vert = vertical_spat_grad(data5m)
    
    # Horizontal Climate Velocity
    #average to yearly timestep
    data_year = data.groupby('time.year').mean('time')
    
    data_yr = inSHAPE_all_depth_glorys(data_year.longitude, data_year.latitude, data_year.thetao[:,:2,:,:], data_year.depth[:2],
                             LME[LME.LME_NAME == LME_Name].geometry[LME.OBJECTID[LME.LME_NAME == LME_Name].values[0] -1])
    datayr = xr.Dataset(data_vars = {'temp': (['year','lat','lon'],data_yr[:,0,:,:])}, 
                   coords = {'year': data_year.year, 'lat': np.array(data_year.latitude), 'lon': np.array(data_year.longitude)})
    
    #calculate the temperature trend
    s, p = temptrend_hcv(datayr)
    
    #calculate the horizontal spatial gradient
    ew_sg, ns_sg, sg = horiz_spatgrad_hcv(datayr)
    
    #calculate the climate velocity
    vcv, hcv = VCV_HCV(spatgrad_vert,slopes,pvals,data5m, datayr, s, p, sg, ew_sg, ns_sg)
    
    vcv.to_netcdf(directory + file_name + 'vcv.nc')
    hcv.to_netcdf(directory + file_name + 'hcv.nc')